# Import configuration and setup

In [1]:
from medulloblastoma.config import PROJ_ROOT, RAW_DATA_DIR, INTERIM_DATA_DIR, PROCESSED_DATA_DIR, EXTERNAL_DATA_DIR, MODELS_DIR, REPORTS_DIR, FIGURES_DIR
import os
import numpy as np
import pandas as pd
os.chdir(PROJ_ROOT)
from medulloblastoma.dataset import download_data, prepare_data
from medulloblastoma.features import main as preprocess_pipeline
from medulloblastoma.features import load_data
# Execute R script for GSE85217 data download
# ! Rscript {os.path.join(PROJ_ROOT, 'medulloblastoma','get_data.R')}

2025-12-12 11:38:08.753 | INFO     | medulloblastoma.config:<module>:11 - PROJ_ROOT path is: /mnt/d/Repos/bitsxlamarato-medulloblastoma
2025-12-12 11:38:15.615482: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-12 11:38:15.621734: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-12 11:38:15.633517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765535895.653792  912282 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765535895.659408  912282 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000

# Prepare Data

In [2]:
# Download microarray gene expression data
download_data(save_path=RAW_DATA_DIR,remove_gz=True)

2025-12-12 11:38:20.158 | INFO     | medulloblastoma.dataset:download_data:77 - Downloading GSE85217 dataset from GEO...
Downloading: 100.0%
File downloaded successfully: /mnt/d/Repos/bitsxlamarato-medulloblastoma/data/raw/GSE85217_M_exp_763_MB_SubtypeStudy_TaylorLab.txt.gz
2025-12-12 11:38:43.395 | INFO     | medulloblastoma.dataset:download_data:96 - Extracting compressed file...
2025-12-12 11:38:51.770 | SUCCESS  | medulloblastoma.dataset:download_data:101 - File extracted to: /mnt/d/Repos/bitsxlamarato-medulloblastoma/data/raw/GSE85217_M_exp_763_MB_SubtypeStudy_TaylorLab.txt
2025-12-12 11:38:51.790 | INFO     | medulloblastoma.dataset:download_data:106 - Removed compressed .gz file


'/mnt/d/Repos/bitsxlamarato-medulloblastoma/data/raw/GSE85217_M_exp_763_MB_SubtypeStudy_TaylorLab.txt'

In [3]:
# Structure data so that it is easier to handle
prepare_data(
    expression_file=os.path.join(RAW_DATA_DIR,'GSE85217_M_exp_763_MB_SubtypeStudy_TaylorLab.txt'),
    metadata_path = os.path.join(RAW_DATA_DIR,'GSE85217_metadata.csv'),
    save_path=RAW_DATA_DIR
)

2025-12-12 11:38:51.803 | INFO     | medulloblastoma.dataset:prepare_data:178 - Loading gene expression data from /mnt/d/Repos/bitsxlamarato-medulloblastoma/data/raw/GSE85217_M_exp_763_MB_SubtypeStudy_TaylorLab.txt
2025-12-12 11:38:56.555 | INFO     | medulloblastoma.dataset:prepare_data:183 - Loaded expression data with shape: (21641, 767)
2025-12-12 11:38:56.557 | INFO     | medulloblastoma.dataset:prepare_data:188 - Extracting gene correspondence information...
2025-12-12 11:38:56.631 | INFO     | medulloblastoma.dataset:prepare_data:192 - Expression data after removing gene info: (21641, 763)
2025-12-12 11:38:56.631 | INFO     | medulloblastoma.dataset:prepare_data:195 - Loading metadata from /mnt/d/Repos/bitsxlamarato-medulloblastoma/data/raw/GSE85217_metadata.csv
2025-12-12 11:38:56.657 | INFO     | medulloblastoma.dataset:prepare_data:200 - Loaded metadata with shape: (763, 37)
2025-12-12 11:38:56.657 | INFO     | medulloblastoma.dataset:prepare_data:203 - Matching sample names 

('/mnt/d/Repos/bitsxlamarato-medulloblastoma/data/raw/cavalli.csv',
 '/mnt/d/Repos/bitsxlamarato-medulloblastoma/data/raw/cavalli_subgroups.csv')

# Preprocessing

Selecting genes that are lowly expressed, lowly variant, and outlier genes. We also check that there are no missing data.

In [4]:
preprocess_pipeline(
    data_path=os.path.join(RAW_DATA_DIR,'cavalli.csv'),
    metadata_path=os.path.join(RAW_DATA_DIR,'cavalli_subgroups.csv'),
    save_path=PROCESSED_DATA_DIR,
    per=0.2,
    cutoff=0.1,
    alpha=0.05
)

Path to save data: /mnt/d/Repos/bitsxlamarato-medulloblastoma/data/processed
2025-12-12 11:39:25.116 | INFO     | medulloblastoma.features:load_data:84 - Loading data from /mnt/d/Repos/bitsxlamarato-medulloblastoma/data/raw/cavalli.csv
2025-12-12 11:39:29.511 | INFO     | medulloblastoma.features:load_data:87 - Loading metadata from /mnt/d/Repos/bitsxlamarato-medulloblastoma/data/raw/cavalli_subgroups.csv
2025-12-12 11:39:29.518 | INFO     | medulloblastoma.features:load_data:91 - Initial data shape: (21641, 763), metadata shape: (763,)
2025-12-12 11:39:29.518 | INFO     | medulloblastoma.features:load_data:94 - Transposing data to match metadata dimensions
2025-12-12 11:39:29.518 | SUCCESS  | medulloblastoma.features:load_data:106 - Successfully loaded data: (763, 21641) samples × 21641 features
Shape of original data: (763, 21641)
null in data: 0
na in data: 0
Are all values in the data real? Yes
2025-12-12 11:39:32.694 | INFO     | medulloblastoma.features:get_g3g4:148 - Extracted 4

<Figure size 700x500 with 0 Axes>

<Figure size 700x500 with 0 Axes>

<Figure size 700x500 with 0 Axes>

<Figure size 700x500 with 0 Axes>

In [5]:
# Load preprocessed gene expression data
data=pd.read_csv(os.path.join(PROCESSED_DATA_DIR,'cavalli_maha.csv'),index_col=0)

In [6]:
# Load metadata
metadata=pd.read_csv(os.path.join(RAW_DATA_DIR,'cavalli_subgroups.csv'),index_col=0).squeeze()
# Adapt names of groups of interests so they are shorter
metadata=metadata.map({'Group3':'G3','Group4':'G4'})
metadata.name = 'Subgroup'
# Select groups of interest in metedata
metadata_g3g4=metadata[metadata.isin(['G3','G4'])]
print(metadata.shape,metadata_g3g4.shape)

(763,) (470,)


In [7]:
# Select groups of interest in gene expression data
data_g3g4 = data.loc[metadata_g3g4.index]
print(data_g3g4.shape)

(470, 14347)


In [8]:
metadata.value_counts()

Subgroup
G4    326
G3    144
Name: count, dtype: int64

In [9]:
metadata_g3g4.value_counts()

Subgroup
G4    326
G3    144
Name: count, dtype: int64

In [10]:
# # Load final datasets
# data,metadata=load_data(
#     data_path=os.path.join(PROCESSED_DATA_DIR,'cavalli_maha.csv'),
#     metadata_path=os.path.join(PROCESSED_DATA_DIR,'g3g4_maha.csv')
# )
# print(data.shape,metadata.shape)

# UMAP Visualization of Preprocessed Data

In [11]:
from medulloblastoma.plots import plot_umap_binary

# Discrete color mapping for G3/G4 subtypes
dict_medulloblastoma = {
    'G3': 'red',  # Red for G3
    'G4': 'blue'   # Blue for G4
}

# Generate UMAP with discrete subtype coloring
plot_umap_binary(
    data=data_g3g4,
    clinical=metadata_g3g4,
    colors_dict=dict_medulloblastoma,
    n_components=2,
    save_fig=True,
    save_as="initial_medulloblastoma_umap",
    seed=2023,
    title="Medulloblastoma G3/G4 Gene Expression UMAP",
    marker_size=20
)

X_umap.shape (470, 2)
color_series.shape (470,)
len(all_patients) 470
color_series.loc[all_patients].values.shape (470,)
Saved UMAP plotly figure to: initial_medulloblastoma_umap.png
Saved UMAP plotly figure to: initial_medulloblastoma_umap.pdf
Saved UMAP plotly figure to: initial_medulloblastoma_umap.svg


# Model Training and Reconstruction

In [12]:
# BLANK SECTION FOR MODEL TRAINING
# This section will be implemented during the hackathon
# Plan for:
# - CVAE architecture definition
# - Training loop with G3/G4 labels
# - Hyperparameter optimization
# - Model evaluation and validation
# Clue: for architecture and hyperparameter optimization, you can use ax (see https://ax.dev/)

# Final UMAP with Continuous Scoring

In [13]:
from medulloblastoma.plots import plot_umap_spectrum

In [14]:
# For testing purposes, we assign a random score between 0 and 1 to each patient
score=pd.Series(np.random.rand(470),name='score',index=metadata_g3g4.index)

In [15]:
plot_umap_spectrum(
    data=data_g3g4,
    clinical=score,  # Continuous scores instead of discrete labels
    colormap='RdBu',
    n_components=2,
    save_fig=True,
    save_as="final_scored_medulloblastoma_umap",
    seed=2023,
    title="Medulloblastoma G3/G4 Scores",
    marker_size=20
)

X_umap.shape (470, 2)
color_series.shape (470,)
color_series range: [0.001, 1.000]
len(all_patients) 470
Saved UMAP spectrum figure to: final_scored_medulloblastoma_umap.png
Saved UMAP spectrum figure to: final_scored_medulloblastoma_umap.pdf
Saved UMAP spectrum figure to: final_scored_medulloblastoma_umap.svg
